# TFIDF full score approach
**to overcome knowing the n° of clusters**

In [ ]:
import pandas as pd
import nltk
import numpy as np
import zipfile
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import re



In [ ]:
!pip install unidecode
from unidecode import unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
INPUT_EXTRACTION = "/content/change_request (7).xlsx"#"/content/estrazione_chg_emergency.xlsx"

In [ ]:
"""ZIPPING FUNCTION"""

def zip_it_multiple_files():
    excel_dir = '/content/excel_files/'

    # Iterate over the DataFrame list and save each DataFrame as an Excel file
    for i, df in enumerate(df_list_labels_per_machine):
        # Construct the directory path based on the machine name
        machine_name = list_of_machines_ordered[i]
        machine_dir = os.path.join(excel_dir, machine_name)

        # Ensure that the directory exists, creating it if necessary
        os.makedirs(machine_dir, exist_ok=True)

        # Construct the Excel file name
        excel_file_name = f"{machine_name.replace('/', '_')}.xlsx"  # Replace '/' with another character
        excel_file_path = os.path.join(machine_dir, excel_file_name)

        # Save the DataFrame to the Excel file
        df.to_excel(excel_file_path, index=False)

    # Create a zip file containing all Excel files
    with zipfile.ZipFile('/content/data.zip', 'w') as zipf:
        for root, dirs, files in os.walk(excel_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, arcname=file)

    # Download the zip file (Colab specific)
    from google.colab import files
    files.download('/content/data.zip')

# Preprocessing

In [ ]:
nltk.download('stopwords')
#dove viene usato punkt?
nltk.download('punkt')
sequence_to_remove = ['descrizione del problema', 'analisi e soluzione','modifica','emergenza','intervento','eseguito','produzione','appeso','-', 'EMERGENCY', 'DATA', 'SUPPORT', 'MICS-SHARED', 'macchina', 'PA', '"Linea', 'Modifica', 'MI&CS', 'di', 'Filling', 'Problema', 'su', 'MICS', 'PFS', 'per', 'SESTO', 'Esecuzione', 'Emergency','<']
stop_words = set(nltk.corpus.stopwords.words('italian') + [])

def data_cleaning_bow(dataframe):
    def text_cleaning(text_raw):
        text_filtered = text_raw.lower()  # lowercase
        # remove text after '***Dati Modificati***'
        text_filtered = re.sub('\n.*'+'dati modificati'+'.*', ' ', text_filtered)
        # remove section separators
        for el in sequence_to_remove:
            #text_filtered = re.sub('\n.*'+el+'.*\n', ' ', text_filtered)
            text_filtered = re.sub(r'\b{}\b'.format(re.escape(el)), '', text_filtered)
        # clean and standardize the text
        text_filtered = unidecode(text_filtered)  # remove accents
        text_filtered = re.sub('[^A-Za-z0-9_ ]+', ' ', text_filtered)
        #why tokenize now ? Does it interacts with the following tokenization of the vectorizer?
        words = nltk.word_tokenize(text_filtered, language="italian")  # tokenize
        filtered_sentence = [w for w in words if w not in stop_words]  # remove stopwords
        return filtered_sentence

    def extract_bag_of_words(row):
        input_text = row["Short Description"] + " " + row["Description"]
        input_cleaned = text_cleaning(input_text)
        return ' '.join(input_cleaned)

    dataframe["BoW"] = dataframe.apply(extract_bag_of_words, axis=1)
    return dataframe

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Vectorization

In [ ]:
def train_tf_idf(dataframe):
    # Initialize the vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the corpus
    tfidf_matrix = vectorizer.fit_transform(list(dataframe['BoW']))

    return vectorizer, tfidf_matrix


def get_test_scores(vectorizer,matrix_bucket):
    #sentence = [sentence]
    tfidf_matrix_bucket = vectorizer.transform(matrix_bucket)
    scores = cosine_similarity(tfidf_matrix_bucket, tfidf_matrix_bucket)
    scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
    return scores_normed

def eliminate_duplicates(df):
    # Sort the groups based on the length of unique 'Number' values to ensure larger groups are compared first.
    # This makes it easier to identify subsets since a larger group cannot be a subset of a smaller one.
    grouped_df = df.groupby('labels', as_index=False)
    groups = [group for _, group in grouped_df]
    groups.sort(key=lambda x: len(set(x['Number'])), reverse=True)

    to_remove = []
    for i in range(len(groups) - 1):
        if i in to_remove:
            continue
        for j in range(i + 1, len(groups)):
            if j in to_remove:
                continue
            set_i = set(groups[i]['Number'])
            set_j = set(groups[j]['Number'])

            # If set_j is a subset of set_i, mark set_j for removal
            if set_j.issubset(set_i):
                to_remove.append(j)
            # Alternatively, if set_i is a subset of set_j, mark set_i for removal
            elif set_i.issubset(set_j):
                to_remove.append(i)
                break  # No need to compare i with other groups if it's already marked for removal

    # Remove identified subsets
    final_groups = [group for index, group in enumerate(groups) if index not in to_remove]

    # Concatenate the remaining groups
    result_df = pd.concat(final_groups).reset_index(drop=True)
    return result_df


def clusterize(df_with_labels, df_clean, scores, similarity_threshold):
    # Assuming 'scores' is defined outside this function and is accessible here.
    # 'scores' should be a 2D array or DataFrame of similarity scores between rows.

    df_clean['labels'] = 'Gruppo_0'  # Initialize all labels to a default group
    cluster_label_counter = 0  # Initialize cluster counter

    for i in range(len(df_clean)):
        similar_indices = np.where(scores[:, i] > similarity_threshold)[0]
        if len(similar_indices) > 0:
            cluster_label = f'label_{cluster_label_counter}'
            df_clean.loc[similar_indices, 'labels'] = cluster_label
            cluster_label_counter += 1  # Increment for a new cluster label in the next iteration

    # Now, all entries have been assigned to clusters based on similarity.
    # Concatenate with df_with_labels if needed (assuming df_with_labels is initially empty or needs updating).
        df_with_labels = pd.concat([df_with_labels, df_clean.loc[similar_indices].drop('BoW', axis=1)], ignore_index=True)

    # Before removing duplicates, let's see the size
    print(f"BEFORE DUPLICATES REMOVAL {df_with_labels.shape[0]}")
    df_with_labels = eliminate_duplicates(df_with_labels)  # Assuming this function is optimized as discussed.
    print(f"END NUMBER ELEMENTS {df_with_labels.shape[0]}")

    return df_with_labels

# Maybe use panel to portray a report of most frequent configuration items

In [ ]:
#def main():
df = pd.read_excel(INPUT_EXTRACTION,sheet_name='Page 1')
list_of_machines_ordered = list(df['Configuration Item'].value_counts().index)
similarity_threshold = 0.7
#Create a list containing the various df's, one for each configuration item
df_list_per_machine = []
#Create list of df's with labels, one for each configuration item
df_list_labels_per_machine = []
for i, machine_name in enumerate(list_of_machines_ordered):
    #if i == 0:
     # continue
    df_filtered = df[df['Configuration Item'] == machine_name].copy(deep=True).reset_index(drop = True)
    df_list_per_machine.append(df_filtered)

#index to keep track of how many times we didn't even had the risk of having duplicates
list_of_elimination_dulicates= []
i= 0
for df_raw in df_list_per_machine:
  i+=1
  df_clean = data_cleaning_bow(df_raw)
  #Instantiate a new dataframe that will add the labels to each row
  df_with_labels = pd.DataFrame(columns = df_clean.columns)
  #clusters = {}
  #Initialize Vectorizer fitted for each ci's vocabulary only
  vectorizer = TfidfVectorizer().fit(list(df_clean['BoW']))
  #Compute diverse scores, generating an nxn matrix. Each column represents an individual emergency (i-th), while the rows below it display the cosine similarity scores with respect to the j-th emergency
  scores = get_test_scores(vectorizer, list(df_clean['BoW']))
  #populate the new dataframe with the relevant labels
  print(f"Machine {i}")
  print(f"BEFORE CLUSTERING{df_clean.shape[0]}")
  if df_clean.shape[0] == 1:
    df_with_labels = df_clean.copy(deep=True).reset_index(drop = True)
    df_with_labels['labels'] = 'Gruppo_0'
  else:
    if i == 1:
      df_with_labels = clusterize(df_with_labels,df_clean,scores,0.825)
    else:
      df_with_labels = clusterize(df_with_labels,df_clean,scores,similarity_threshold)
  df_list_labels_per_machine.append(df_with_labels)



df_only = pd.concat(df_list_labels_per_machine,ignore_index=True)
df_only.to_excel('excel_label_removed_duplicates.xlsx')

# Call the function to ZIP
#zip_it_multiple_files()
#main()





/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Machine 1
BEFORE CLUSTERING99
BEFORE DUPLICATES REMOVAL 143
END NUMBER ELEMENTS 99
Machine 2
BEFORE CLUSTERING89
BEFORE DUPLICATES REMOVAL 199
END NUMBER ELEMENTS 128
Machine 3
BEFORE CLUSTERING67
BEFORE DUPLICATES REMOVAL 151
END NUMBER ELEMENTS 70
Machine 4
BEFORE CLUSTERING60
BEFORE DUPLICATES REMOVAL 94
END NUMBER ELEMENTS 60
Machine 5
BEFORE CLUSTERING52
BEFORE DUPLICATES REMOVAL 64
END NUMBER ELEMENTS 52
Machine 6
BEFORE CLUSTERING48
BEFORE DUPLICATES REMOVAL 144
END NUMBER ELEMENTS 48
Machine 7
BEFORE CLUSTERING47
BEFORE DUPLICATES REMOVAL 63
END NUMBER ELEMENTS 47
Machine 8
BEFORE CLUSTERING45
BEFORE DUPLICATES REMOVAL 61
END NUMBER ELEMENTS 45
Machine 9
BEFORE CLUSTERING35
BEFORE DUPLICATES REMOVAL 37
END NUMBER ELEMENTS 35
Machine 10
BEFORE CLUSTERING32
BEFORE DUPLICATES REMOVAL 44
END NUMBER ELEMENTS 32
Machine 11
BEFORE CLUSTERING28
BEFORE DUPLICATES REMOVAL 30
END NUMBER ELEMENTS 28
Machine 12
BEFORE CLUSTERING27
BEFORE DUPLICATES REMOVAL 35
END NUMBER ELEMENTS 27
Machine 

<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores)

Machine 58
BEFORE CLUSTERING1
Machine 59
BEFORE CLUSTERING1
Machine 60
BEFORE CLUSTERING1
Machine 61
BEFORE CLUSTERING1
Machine 62
BEFORE CLUSTERING1
Machine 63
BEFORE CLUSTERING1
Machine 64
BEFORE CLUSTERING1
Machine 65
BEFORE CLUSTERING1
Machine 66
BEFORE CLUSTERING1


<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores)

Machine 67
BEFORE CLUSTERING1
Machine 68
BEFORE CLUSTERING1
Machine 69
BEFORE CLUSTERING1
Machine 70
BEFORE CLUSTERING1


<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
<ipython-input-6-2aed3fa7ad08>:15: RuntimeWarning: invalid value encountered in divide
  scores_normed = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))


In [ ]:
df_list_labels_per_machine[49]

,Number,Short Description,Description,Opened,Configuration Item,Closed,BoW,labels
0,CHG1686242,MI&CS PA - EMERGENCY - CERES - XFPMAQW12FL - E...,***** DESCRIZIONE DEL PROBLEMA *****\nDurante ...,2020-12-08 00:09:15,SESTO Palltronic SCADA AquaWIT - PRD 1.0,2021-01-07 08:55:23,NaN,label_0
1,CHG1686765,"MI&CS PA - EMERGENCY - CERES - XFPMAQW12FL, XF...",***** DESCRIZIONE DEL PROBLEMA *****\nDurante ...,2020-12-09 00:17:47,SESTO Palltronic SCADA AquaWIT - PRD 1.0,2021-01-11 09:26:20,NaN,label_1


In [ ]:
list1 = ['CHG2038328', 'CHG2272738', 'CHG2289064', 'CHG2342485']
list2 = ['CHG2279054', 'CHG2279656','CHG2289064']

res3 = all([ele in list1 for ele in list2])

In [ ]:
print(len([i for i in list_of_elimination_dulicates if i == False]))

0


In [ ]:
"""OLD ALGORITHM"""
# def eliminate_duplicates(labels, df):
#     df_list = []
#     for label in labels:
#         df_lab = df[df['labels'] == label].copy(deep=True).reset_index(drop = True)
#         df_list.append(df_lab)
#     bad_indexes = []
#     for i in range(len(df_list)):
#       chg_i = df_list[i]['Number'].values
#       for j in range(len(df_list)):
#         if i!=j:
#           chg_j = df_list[j]['Number'].values
#           is_matching = True
#           #qui non va bene(immaginiamo il caso che chg_j sia più piccolo ma sia contenuto in chg_i...non verrebbe eliminato).
#           #La soluzione è riscrivere il ciclo di modo che sia possibile controllare anche in direzione contraria(l'ottimizazzione che si perde è un fattore costante..vedi gauss)
#           if len(chg_i) > len(chg_j):
#             continue
#           for el in chg_i:
#             #questo deve verificarsi per ogni chg_j..altrimenti vuol dire che chg_i è duplicato da qualcuno
#             if not (el in chg_j ):
#               is_matching = False
#               break
#           if is_matching:
#             bad_indexes.append(i)
#             #added this break since the previous brake only exited the first inner loop 'for el in chg_i'
#             break

#     result_df = [df_list[i] for i in range(len(df_list)) if i not in bad_indexes]
#     if len(result_df)>1:
#       result_df = pd.concat(result_df)
#     else:
#       result_df = result_df[0]

#     return result_df


'OLD ALGORITHM'

In [ ]:
# def eliminate_duplicates(labels, df):
#     df_list = []
#     for label in labels:
#         df_lab = df[df['labels'] == label].copy(deep=True).reset_index(drop = True)
#         df_list.append(df_lab)
#     bad_indexes = []
#     for i in range(len(df_list)-1):
#       chg_i = list(df_list[i]['Number'].values)
#       for j in range(i+1,len(df_list)):
#         #if i!=j:
#         chg_j = list(df_list[j]['Number'].values)
#         if len(chg_j)>=len(chg_i):
#           res_i = all([ele in chg_j for ele in chg_i])
#           res_j = False
#         else:
#           res_j = all([ele in chg_i for ele in chg_j])
#           res_i = False
#         if res_i:
#           bad_indexes.append(i)
#           continue
#         elif res_j:
#           bad_indexes.append(j)



#     result_df = [df_list[i] for i in range(len(df_list)) if i not in set(bad_indexes)]
#     if len(result_df)>1:
#       result_df = pd.concat(result_df)
#     else:
#       result_df = result_df[0]
#     return result_df

In [ ]:
# def clusterize(df_with_labels,df_clean):
#   clusters = {}
#   #Initialize labels column (Poteva essere un problema crearlo on the spot come prima?)
#   df_clean['labels'] = 0
#   for i in range(len(list(df_clean['BoW']))):
#     similar_indices = np.argwhere(scores[:, i] > similarity_threshold)
#     #It gives me back a list of lists where each list contains a single element being the row where the score was higher or equal to similarity score
#     similar_indices = [indices[0] for indices in similar_indices ] #I take only the first one
#     # Create a new cluster with the query sentence and similar sentences
#     cluster_label = f'Gruppo_{i}'
#     clusters[cluster_label] = [i]  # Add the query sentence index to the cluster
#     df_clean.loc[i, 'labels'] = cluster_label # Add the label to the query sentence
#     for idx in similar_indices:
#       if idx != i:  # Exclude the query sentence itself
#         clusters[cluster_label].append(idx)
#         df_clean.loc[idx, 'labels'] = cluster_label  # Add the label to the similar sentences
#     df_with_labels = pd.concat([df_with_labels,df_clean.loc[similar_indices]], ignore_index=True)
#     df_with_labels.drop('BoW',axis=1,inplace= True)
#   print(f"BEFORE DUPLICATES REMOVAL {df_with_labels.shape[0]}")
#   df_with_labels = eliminate_duplicates(df_with_labels)
#   print(f"END NUMBER ELEMENTS {df_with_labels.shape[0]}")
#   print("////////\n///////")
#   return df_with_labels